In [ ]:
def compare_labels(product_a: dict, product_b: dict) -> dict:
    """
    Compare nutrient facts between two food products.

    Args:
        product_a (dict): Nutrient data for product A. Keys are nutrient names, values are amounts.
        product_b (dict): Nutrient data for product B. Same format as product_a.

    Returns:
        dict: A dictionary with nutrient names as keys and a tuple of (product_a_value, product_b_value, comparison)
              where comparison is 'higher in A', 'higher in B', or 'equal'.

    Raises:
        ValueError: If either input is not a dictionary or missing nutrient keys.
    """
    if not isinstance(product_a, dict) or not isinstance(product_b, dict):
        raise ValueError("Both inputs must be dictionaries containing nutrient data.")

    comparison_result = {}
    common_nutrients = set(product_a.keys()) & set(product_b.keys())

    for nutrient in common_nutrients:
        a_val = product_a.get(nutrient, 0)
        b_val = product_b.get(nutrient, 0)

        if not isinstance(a_val, (int, float)) or not isinstance(b_val, (int, float)):
            raise ValueError(f"Nutrient values must be numeric for '{nutrient}'.")

        if a_val > b_val:
            comparison = "higher in A"
        elif b_val > a_val:
            comparison = "higher in B"
        else:
            comparison = "equal"

        comparison_result[nutrient] = (a_val, b_val, comparison)

    return comparison_result

In [ ]:
def search_keyword(text: str, keyword: str) -> bool:
    """
    Search for a keyword in a block of text.

    Args:
        text (str): The text to search within.
        keyword (str): The keyword to search for.

    Returns:
        bool: True if keyword is found (case-insensitive), False otherwise.

    Raises:
        ValueError: If either input is not a string.
    """
    if not isinstance(text, str) or not isinstance(keyword, str):
        raise ValueError("Both text and keyword must be strings.")

    return keyword.lower() in text.lower()

In [ ]:
def calculate_nutri_score_letter(nutrients: dict) -> str:
    """
    Calculates Nutri-Score letter grade (A to E) based on nutrient profile.
    Args:
        nutrients (dict): Nutrient values with keys like 'sugar', 'saturated_fat', 'sodium', 'fiber', 'protein', 'fruits_veg_percent'
    Returns:
        str: Nutri-Score letter ('A' to 'E')
    Raises:
        ValueError: If required nutrients are missing or invalid
    """
    try:
        sugar = float(nutrients.get('sugar', 0))
        sat_fat = float(nutrients.get('saturated_fat', 0))
        sodium = float(nutrients.get('sodium', 0))
        fiber = float(nutrients.get('fiber', 0))
        protein = float(nutrients.get('protein', 0))
        fruits_veg = float(nutrients.get('fruits_veg_percent', 0))
    except (TypeError, ValueError):
        raise ValueError("Invalid nutrient values provided.")

    # Negative points
    negative = sugar * 2 + sat_fat * 3 + sodium * 1.5

    # Positive points
    positive = fiber * 2 + protein * 1.5 + fruits_veg * 2

    score = round(negative - positive)

    # Map score to Nutri-Score letter
    if score <= -1:
        return 'A'
    elif score <= 2:
        return 'B'
    elif score <= 10:
        return 'C'
    elif score <= 18:
        return 'D'
    else:
        return 'E'

In [ ]:
from typing import List, Dict
from html.parser import HTMLParser

def format_search_results(results: List[Dict[str, str]]) -> List[Dict[str, str]]:
    """
    Clean and format a list of search result entries containing brand and product names.

    Each entry must include 'brand' and 'product'. This function:
    - Removes HTML tags from both fields
    - Filters out special characters except basic punctuation
    - Returns only entries with valid, readable brand and product names

    Args:
        results (List[Dict[str, str]]): List of search result dictionaries.

    Returns:
        List[Dict[str, str]]: Cleaned and validated search results.

    Raises:
        ValueError: If input is not a list of dictionaries.
    """

    class HTMLCleaner(HTMLParser):
        def __init__(self):
            super().__init__()
            self.text_parts = []

        def handle_data(self, data):
            self.text_parts.append(data)

        def get_clean_text(self):
            return ''.join(self.text_parts).strip()

    def clean_text(text: str) -> str:
        if not isinstance(text, str):
            raise ValueError("Text must be a string.")
        parser = HTMLCleaner()
        parser.feed(text)
        raw = parser.get_clean_text()
        allowed = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,!? -"
        return ''.join(c for c in raw if c in allowed).strip()

    if not isinstance(results, list):
        raise ValueError("Input must be a list of dictionaries.")

    formatted = []
    for entry in results:
        if not all(k in entry for k in ('brand', 'product')):
            continue

        brand = clean_text(entry['brand'])
        product = clean_text(entry['product'])

        if brand and product:
            formatted.append({'brand': brand, 'product': product})

    return formatted